## Investigating problems closing a deposit account

GitHub: https://github.com/MkershMambu/MambuAPINotebook/blob/master/CloseDepositAccount.ipynb

Viewer: [https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/CloseDepositAccount.ipynb](https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/CloseDepositAccount.ipynb?flush_cache=true)

The below is my investigation to duplicate the problem, attempt to find a way to do it with API 2.0 and finally to the solution using an API 1.0 functions.

**To jump straight to the solution [Click Here](#solution)**

In [177]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

### First lets create a new deposit account

In [131]:
%%writefile JSONBODY
{
  "overdraftSettings": {
	"allowOverdraft": true,
	"overdraftLimit": 0
  },
  "accountType": "CURRENT_ACCOUNT",
  "name": "MKCurTest222",
  "accountHolderKey": "8a8186ac692678910169288509c606bc",
  "productTypeKey": "8a8186aa69491c25016949888aeb10d2",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT"
}

Overwriting JSONBODY


In [132]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)
encodedKey = r.json()['encodedKey']

API status:
201
Now the JSON:
{"encodedKey":"8a8186e36a167f22016a16dce3e70032","creationDate":"2019-04-13T15:22:16+02:00","lastModifiedDate":"2019-04-13T15:22:16+02:00","id":"MEUK590","name":"MKCurTest222","accountHolderType":"CLIENT","accountHolderKey":"8a8186ac692678910169288509c606bc","accountState":"PENDING_APPROVAL","productTypeKey":"8a8186aa69491c25016949888aeb10d2","accountType":"CURRENT_ACCOUNT","currencyCode":"EUR","assignedBranchKey":"8a8186ac692678910169287cf43606af","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a8186e36a167f22016a16dce3e70033","interestRate":0E-20,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestRateSource":"FIXED_INTEREST_RATE"}},"balances":{"overdraftAmount":0,"technicalOverdraftAmount":0,"locke

### Deposit money into the account

In [138]:
%%writefile JSONBODY
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 500,
  "notes": "some notes",
  "paymentOrderId": "{{UniqueID}}",
  "externalId": "{{UniqueID}}"
}

Overwriting JSONBODY


In [139]:

HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
BODYPARTS = {'UniqueID': getUUID()}
url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY", bodyparts = BODYPARTS )
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":407,"errorReason":"INVALID_DEPOSIT_ACCOUNT_STATE"}]}


### We need to approve the deposit account before we can deposit into it

In [140]:
%%writefile JSONBODY
{
  "action": "APPROVE",
  "notes": "more notes"
}

Overwriting JSONBODY


In [141]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a8186e36a167f22016a16dce3e70032","creationDate":"2019-04-13T15:22:16+02:00","lastModifiedDate":"2019-04-13T15:30:31+02:00","id":"MEUK590","name":"MKCurTest222","accountHolderType":"CLIENT","accountHolderKey":"8a8186ac692678910169288509c606bc","accountState":"APPROVED","productTypeKey":"8a8186aa69491c25016949888aeb10d2","accountType":"CURRENT_ACCOUNT","approvedDate":"2019-04-13T15:30:31+02:00","currencyCode":"EUR","assignedBranchKey":"8a8186ac692678910169287cf43606af","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0E-10},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a8186e36a167f22016a16dce3e70033","interestRate":0E-20,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestRateSource":"FIXED_INTEREST_RATE"}},"balances":{"overdraftAmount

### Now we should be able to deposit

In [142]:
%%writefile JSONBODY
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 500,
  "notes": "some notes",
  "paymentOrderId": "{{UniqueID}}",
  "externalId": "{{UniqueID}}"
}

Overwriting JSONBODY


In [143]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
BODYPARTS = {'UniqueID': getUUID()}
url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY", bodyparts=BODYPARTS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
201
Now the JSON:
{"encodedKey":"8a81862d6a16e587016a16e587f00001","id":"94","externalId":"090f393f-20be-4f34-8f89-cf609efc0ac2","paymentOrderId":"090f393f-20be-4f34-8f89-cf609efc0ac2","creationDate":"2019-04-13T15:31:43+02:00","valueDate":"2019-04-13T15:31:41+02:00","notes":"some notes","parentAccountKey":"8a8186e36a167f22016a16dce3e70032","type":"DEPOSIT","amount":500,"currencyCode":"EUR","affectedAmounts":{"fundsAmount":500.0000000000,"interestAmount":0,"feesAmount":0,"overdraftAmount":0E-10,"overdraftFeesAmount":0E-10,"overdraftInterestAmount":0E-10,"technicalOverdraftAmount":0E-10,"technicalOverdraftInterestAmount":0E-10,"fractionAmount":0},"taxes":{},"accountBalances":{"totalBalance":500.0000000000},"userKey":"8a8186ac69267891016928db604e0735","branchKey":"8a8186ac692678910169287cf43606af","terms":{"interestSettings":{},"overdraftInterestSettings":{"interestRate":0E-20},"overdraftSettings":{"overdraftLimit":0E-10}},"transactionDetails":{"transactionChannelKey":"8a818e

### Now let's see if we can DELETE

In [144]:

HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":105,"errorReason":"INVALID_ACCOUNT_STATE"}]}


### Get Account in a state where we can DELETE

You can't delete whilst there are funds in the account. So Let's change balance to 0

In [145]:
%%writefile JSONBODY
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 500,
  "notes": "some notes",
  "paymentOrderId": "{{UniqueID}}",
  "externalId": "{{UniqueID}}"
}

Overwriting JSONBODY


In [146]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
BODYPARTS = {'UniqueID': getUUID()}
url = "{{env1}}" + "/deposits/{0}/withdrawal-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY", bodyparts=BODYPARTS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
201
Now the JSON:
{"encodedKey":"8a81862d6a16e587016a16e587f00003","id":"95","externalId":"edad1e05-fc7b-4d2f-942c-12f8e2d2f610","paymentOrderId":"edad1e05-fc7b-4d2f-942c-12f8e2d2f610","creationDate":"2019-04-13T15:33:37+02:00","valueDate":"2019-04-13T15:33:37+02:00","notes":"some notes","parentAccountKey":"8a8186e36a167f22016a16dce3e70032","type":"WITHDRAWAL","amount":-500,"currencyCode":"EUR","affectedAmounts":{"fundsAmount":500.0000000000,"interestAmount":0,"feesAmount":0,"overdraftAmount":0E-10,"overdraftFeesAmount":0,"overdraftInterestAmount":0,"technicalOverdraftAmount":0E-10,"technicalOverdraftInterestAmount":0,"fractionAmount":0},"taxes":{},"accountBalances":{"totalBalance":0E-10},"userKey":"8a8186ac69267891016928db604e0735","branchKey":"8a8186ac692678910169287cf43606af","terms":{"interestSettings":{},"overdraftInterestSettings":{},"overdraftSettings":{}},"transactionDetails":{"transactionChannelKey":"8a818e74677a2e9201677ec2b4c336a6","transactionChannelId":"cash"},

### Now let's see if we can DELETE now

In [147]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":105,"errorReason":"INVALID_ACCOUNT_STATE"}]}


No we can't DELETE. Once account has been approved and had activity on it you can no longer DELETE.
See [https://support.mambu.com/docs/deposit-accounts-life-cycle-and-states](https://support.mambu.com/docs/deposit-accounts-life-cycle-and-states)

We can certainly close from the Mambu UI and this changes "accountState": "CLOSED".

Can we change state to closed using "/deposits/{depositID}:changeState" function?
* No the only actions that you can pass to this are (APPROVE, UNDO_APPROVE, LOCK) See [here](https://api.mambu.com/#tocSdepositaccountaction)
* Can we update the account (with a PUT). No See below

In [178]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
# url = "{{env1}}/deposits/NSUX690"
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET(url, headers=HEADERS, params=PARAMS)

accountDetails = r.json()
accountDetails['accountState'] = "CLOSED"
writeFile(accountDetails,"JSONBODY")

In [179]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = PUT(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":2813,"errorSource":"The account state cannot be changed","errorReason":"DEPOSIT_ACCOUNT_FIELD_NOT_EDITABLE"}]}


### Can we PATCH to make the change

In [180]:
%%writefile JSONBODY
[
  {
    "op": "REPLACE",
    "path": "accountState",
    "value": "CLOSED"
  }
]

Overwriting JSONBODY


In [181]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = PATCH(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":2001,"errorSource":"Field cannot be patched: /accountState","errorReason":"FIELD_NOT_ALLOWED"}]}


Testing that my PATCH works for other attributes

It does I tested with the following:

In [77]:
%%writefile JSONBODY
[
  {
    "op": "REPLACE",
    "path": "name",
    "value": "MKCurTest222YYYYY"
  }
]

Overwriting JSONBODY


### Is there anything in API 1.0 to do this 
[https://support.mambu.com/docs/savings-products-api](https://support.mambu.com/docs/savings-products-api)

Let's to try a DELETE in API 1.0 to see if it behaves differently thena API 2.0

In [182]:
HEADERS = {}
PARAMS = {}
url = "{{env1}}" + "/savings/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"returnCode":416,"returnStatus":"INVALID_SAVINGS_ACCOUNT_STATE_TRANSITION"}


<a id="solution"></a>
### Solution to the problem
A colleague (Rogrigo Ocampo) in my SE team pointed me to the solution:
https://support.mambu.com/docs/loan-transactions-api#post-transactions

You need to use the API 1.0 "/savings/{ID}/transactions" functions


In [183]:
%%writefile JSONBODY
{
    "type": "CLOSE",
    "notes": 'account closed via API'
}


Overwriting JSONBODY


In [184]:
HEADERS = {'Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/savings/{0}/transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a8186e36a167f22016a16dce3e70032","id":"MEUK590","accountHolderKey":"8a8186ac692678910169288509c606bc","accountHolderType":"CLIENT","name":"MKCurTest222","creationDate":"2019-04-13T13:22:16+0000","approvedDate":"2019-04-13T15:30:31+0000","activationDate":"2019-04-13T15:31:41+0000","lastModifiedDate":"2019-04-13T14:02:30+0000","closedDate":"2019-04-13T16:02:29+0000","productTypeKey":"8a8186aa69491c25016949888aeb10d2","accountType":"CURRENT_ACCOUNT","accountState":"CLOSED","balance":"0","accruedInterest":"0","overdraftInterestAccrued":"0","technicalOverdraftInterestAccrued":"0","overdraftAmount":"0","technicalOverdraftAmount":"0","overdraftInterestSettings":{"interestRate":"0","encodedKey":"8a8186e36a167f22016a16dce3e70033","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateSource":"FIXED_INTEREST_RATE","interestRateTerms":"FIXED","accrueInterestAfterMaturity":false},"interestDue":"0","technicalInterestDue":"0